**Task: To develop a Machine Learning Algorithm to predict whether the customer purchases Audio Books again.**

Import Dependencies and Extract data 

In [1]:
import numpy as np
from sklearn import preprocessing

csv_data = np.loadtxt('Audiobooks.csv',delimiter = ',')

unscaled = csv_data[:,1:-1]
targets = csv_data[:,-1]

Balance the dataset

In [2]:
one_targets = int(np.sum(targets))
counter = 0
indices = []

for i in range(targets.shape[0]):
    if targets[i]==0:
        counter+=1
        if counter > one_targets:
            indices.append(i)

unscaled_priors = np.delete(unscaled,indices,axis = 0)
targets_priors = np.delete(targets,indices,axis = 0)

Standardize the inputs

In [3]:
scaled = preprocessing.scale(unscaled_priors)

Shuffle the data

In [4]:
shuffled_indices = np.arange(scaled.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled[shuffled_indices]
shuffled_targets = targets_priors[shuffled_indices]

Splitting of Dataset

In [5]:
sample_count = shuffled_inputs.shape[0]

train = int(0.8*sample_count)
valid = int(0.1*sample_count)
test = sample_count - train - valid

train_inputs = shuffled_inputs[:train]
train_targets = shuffled_targets[:train]

valid_inputs = shuffled_inputs[train:train+valid]
valid_targets= shuffled_targets[train:train+valid]

test_inputs = shuffled_inputs[train+valid:]
test_targets = shuffled_targets[train+valid:]

print(np.sum(train_targets),train,np.sum(train_targets) / train)
print(np.sum(valid_targets), valid, np.sum(valid_targets) / valid)
print(np.sum(test_targets) ,test ,np.sum(test_targets) / test)

1771.0 3579 0.4948309583682593
228.0 447 0.5100671140939598
238.0 448 0.53125


Save the three sets in .npz format

In [6]:
np.savez('Audiobooks_data_train',inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_valid',inputs = valid_inputs, targets = valid_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs , targets = test_targets)

Creating the Machine Learning Algorithm

Importing Libraries

In [7]:
import tensorflow as tf

In [8]:
print(tf.__version__)

2.15.0


In [15]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_valid.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# Model

In [17]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 1s - loss: 0.5853 - accuracy: 0.6829 - val_loss: 0.5028 - val_accuracy: 0.7852 - 1s/epoch - 40ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4659 - accuracy: 0.7687 - val_loss: 0.4129 - val_accuracy: 0.8166 - 186ms/epoch - 5ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4109 - accuracy: 0.7843 - val_loss: 0.3723 - val_accuracy: 0.8277 - 188ms/epoch - 5ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3853 - accuracy: 0.7910 - val_loss: 0.3539 - val_accuracy: 0.8277 - 179ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3707 - accuracy: 0.7994 - val_loss: 0.3451 - val_accuracy: 0.8345 - 173ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3627 - accuracy: 0.7960 - val_loss: 0.3348 - val_accuracy: 0.8367 - 178ms/epoch - 5ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3530 - accuracy: 0.8128 - val_loss: 0.3368 - val_accuracy: 0.8210 - 180ms/epoch - 5ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3487 - accuracy: 0.8066 - val_loss: 0.3274 - val_accuracy: 0.8345 - 190ms/epoch - 5ms/step
Epoch 9/10

# Testing

In [18]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)


14/14 [==============================] - 0s 4ms/step - loss: 0.3456 - accuracy: 0.8103


In [19]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))



Test loss: 0.35. Test accuracy: 81.03%


# Plot